# Spin Hall conductivity

author: Jae-Mo Lihm (jaemo.lihm@gmail.com) and Minsu Ghim (minsu.ghim.physics@gmail.com)

In this tutorial, we calculate the spin Berry curvature and the spin Hall conductivity of bcc Platinum. We compare the two methods for calculating the spin velocity matrix, which we call the "Qiao" method [1] and the "Ryoo" method [2].

[1] [J. Qiao et al, Phys. Rev. B 98, 214402 (2018)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.98.214402) <br>
[2] [J. H. Ryoo et al, Phys. Rev. B 99, 235113 (2019)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.99.235113)

Both methods use the Kubo formula to calculate spin Hall conductivity under time-reversal symmetry:

\begin{equation}
\sigma^{{\rm SHC}, \gamma}_{\alpha\beta} = \frac{-e\hbar}{N_k V_c}\sum_{\bf k}\sum_{n,m}\left(f_{n{\bf k}}-f_{m{\bf k}}\right)\frac{\textrm{Im}\left[\langle\psi_{n{\bf k}}\vert \frac{1}{2}\{ s^{\gamma}, v_\alpha \} \vert\psi_{m{\bf k}}\rangle\langle\psi_{m{\bf k}}\vert v_\beta\vert\psi_{n{\bf k}}\rangle\right]}{(\varepsilon_{n{\bf k}}-\varepsilon_{m{\bf k}})^2-(\hbar\omega+i\eta)^2}\,,
\label{eq:shc}\tag{1}
\end{equation}

where $\alpha$, $\beta$, $\gamma$ are respectively the direction of spin current, applied electric field, and spin polarisation.

The "Ryoo" method requires `.chk`, `.eig`, `.mmn`, `.spn`, `.sHu`, and `.sIu` files to calculate the spin velocity matrix in (\ref{eq:shc}), $\langle\psi_{n{\bf k}}\vert \frac{1}{2}\{ s^{\gamma}, v_\alpha \} \vert\psi_{m{\bf k}}\rangle$,from pw2wannier90.x,
while the "Qiao" method does not use the last two files, and instead applies an approximation $\mathbf{1}=\sum_{l\in \it{ab\,initio}} \vert u_{l{\bf q}}\rangle\langle u_{l{\bf q}}\vert$.
The `sHu` and `sIu` files are calculated by setting `write_sHu = .true.` and `write_sIu = .true.` to the `pw2wannier90.x` input file: see `data_Pt/pw2wan.in`.

In [1]:
# Preliminary (Do only once)
%load_ext autoreload
%autoreload 2

# Set environment variables - not mandatory but recommended
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1' 
os.environ['MKL_NUM_THREADS'] = '1'


import wannierberri as wberri
import numpy as np
import scipy
import matplotlib.pyplot as plt

#  This block is needed if you run this cell for a second time
#  because one cannot initiate two parallel environments at a time
try:
    parallel.shutdown()   
except NameError:
    pass

parallel = wberri.Parallel(num_cpus=4, progress_step_percent=10)

2022-09-05 14:56:35,026	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


### Model, band structure

We load the system from a Wannier90 output. Note the arguments `SHCryoo=True` and `SHCqiao=True` which are required to compute spin Hall conductivity using the Ryoo and Qiao methods, respectively.

We set symmetry using the `set_symmetry_from_structure` method, which calls spglib to automatically determine the symmetry of the system.

<!-- We also symmetrize the system. See `data_Pt/Pt.win` file and check that the initial projections are correct. For details, refer to the symmetrization tutorial. -->

In [2]:
system = wberri.System_w90("data_Pt/Pt", berry=True, SHCryoo=True, SHCqiao=True)
system.set_structure([[0., 0., 0.]], ["Pt"])
system.set_symmetry_from_structure()

efermi = 18.1605

using fortio to read


FileNotFoundError: [Errno 2] No such file or directory: '/home/runner/work/wannier-berri-org/wannier-berri-org/source/tutorials/spin_hall/data_Pt/Pt.chk'

In [3]:
path = wberri.Path(
    system,
    k_nodes=[
        [0.25, 0.75, 0.50], # W
        [0.50, 0.50, 0.50], # L
        [0.00, 0.00, 0.00], # Gamma
        [0.50, 0.00, 0.50], # X
        [0.50, 0.25, 0.75], # W
        [0.00, 0.00, 0.00], # Gamma
    ],
    labels=["W", "L", "$\Gamma$", "X", "W", "$\Gamma$"],
    length=300,
)

from wannierberri import calculators as calc
calculators = {}
calculators["tabulate"] = calc.TabulatorAll(
    {"Energy": calc.tabulate.Energy()},
    ibands=np.arange(system.num_wann),
    mode="path",
)

path_result = wberri.run(
    system,
    grid=path,
    calculators=calculators,
    parallel=parallel,
    print_Kpoints = False,
)

NameError: name 'system' is not defined

In [4]:
fig = path_result.results["tabulate"].plot_path_fat(path, close_fig=False, show_fig=False)

ax = fig.get_axes()[0]
ax.axhline(efermi, c="r", ls="--")
plt.show(fig)

NameError: name 'path_result' is not defined

## Static spin Hall conductivity

We calculate the static (i.e. DC) spin Hall conductivity. We fix $\omega$ to 0 and scan the Fermi energy.

In [5]:
from wannierberri import calculators as calc

efermi_list = np.linspace(efermi - 1.0, efermi + 1.0, 101, True)

kwargs = dict(
    Efermi=efermi_list,
    omega=np.array([0.]),
    smr_fixed_width = 0.1, # Smearing for frequency in eV
    kBT = 0.026, # Smearing for Fermi level (Fermi-Dirac factor) in eV (not Kelvin)
)

calculators = dict(
    SHC_ryoo = calc.dynamic.SHC(SHC_type="ryoo", **kwargs),
    SHC_qiao = calc.dynamic.SHC(SHC_type="qiao", **kwargs),
)

NameError: name 'efermi' is not defined

In [6]:
nk = 30
grid = wberri.Grid(system, NK=nk)
result = wberri.run(
    system,
    grid=grid,
    calculators=calculators,
    parallel=parallel,
    print_Kpoints = False,
)

NameError: name 'system' is not defined

The SHC data has 5 indices:
1. The Fermi level index,
2. The frequency index,
3. The spin current direction index,
4. The electric field direction index, and
5. The spin polarization index.

In [7]:
print("result.results[\"SHC_ryoo\"].data.shape = ", result.results["SHC_ryoo"].data.shape)

NameError: name 'result' is not defined

In [8]:
shc_ryoo = result.results["SHC_ryoo"].data[:, 0, 0, 1, 2]
shc_qiao = result.results["SHC_qiao"].data[:, 0, 0, 1, 2]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(efermi_list, shc_ryoo.real, label="Ryoo")
axes[0].plot(efermi_list, shc_qiao.real, label="Qiao")
axes[1].plot(efermi_list, shc_ryoo.imag)
axes[1].plot(efermi_list, shc_qiao.imag)
for ax in axes:
    ax.set_xlabel("Efermi (eV)")
    ax.axhline(0, c="k")
axes[0].set_ylabel("Re(SHC)")
axes[1].set_ylabel("Im(SHC)")
axes[0].legend()
plt.show()

NameError: name 'result' is not defined

## Dynamic spin Hall conductivity

We calculate the dynamic (i.e. frequency-dependent, AC) spin Hall conductivity. We fix the Fermi energy to the value `efermi` and scan the frequency in the range `omega`.

The `smr_fixed_width` parameter controls the smearing of the frequency-dependent terms (delta functions and principal values).

In [9]:
from wannierberri import calculators as calc

omega = np.linspace(0, 4, 101, True)

kwargs = dict(
    Efermi=np.array([efermi]),
    omega=omega,
    smr_fixed_width = 0.1, # Smearing for frequency in eV
    kBT = 0.026, # Smearing for Fermi level (Fermi-Dirac factor)
)

calculators = dict(
    SHC_ryoo = calc.dynamic.SHC(SHC_type="ryoo", **kwargs),
    SHC_qiao = calc.dynamic.SHC(SHC_type="qiao", **kwargs),
)

NameError: name 'efermi' is not defined

In [10]:
nk = 30
grid = wberri.Grid(system, NK=nk)
result = wberri.run(
    system,
    grid=grid,
    calculators=calculators,
    parallel=parallel,
    print_Kpoints = False,
)

NameError: name 'system' is not defined

In [11]:
shc_ryoo = result.results["SHC_ryoo"].data[0, :, 0, 1, 2]
shc_qiao = result.results["SHC_qiao"].data[0, :, 0, 1, 2]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(omega, shc_ryoo.real, label="Ryoo")
axes[0].plot(omega, shc_qiao.real, label="Qiao")
axes[1].plot(omega, shc_ryoo.imag)
axes[1].plot(omega, shc_qiao.imag)
for ax in axes:
    ax.set_xlabel("omega (eV)")
    ax.axhline(0, c="k")
axes[0].set_ylabel("Re(SHC)")
axes[1].set_ylabel("Im(SHC)")
axes[0].legend()
plt.show()

NameError: name 'result' is not defined

## Spin Berry curvature

To understand the microscopic origin of the spin Hall conductivity, one may inspect the k-resolved spin Berry curvature.
Eq. (\ref{eq:shc}) is recast into the sum of a Berry-curvature-like term, the spin Berry curvature.

The spin Berry curvature is

\begin{equation}
  \Omega^{n, \gamma}_{\alpha\beta}({\bf k}) = -\sum_{m \neq n}\frac{2\textrm{Im}\left[\langle\psi_{n{\bf k}}\vert \frac{1}{2}\{ s^{\gamma}, v_\alpha \} \vert\psi_{m{\bf k}}\rangle\langle\psi_{m{\bf k}}\vert v_\beta\vert\psi_{n{\bf k}}\rangle\right]}{(\varepsilon_{n{\bf k}}-\varepsilon_{m{\bf k}})^2-(i\eta)^2}\,,
  \label{eq:sbc}\tag{2}
\end{equation}

and the k-resolved spin Berry curvature summed over the band index is

\begin{equation}
  \Omega^{\gamma}_{\alpha\beta}({\bf k}) = \sum_{n}f_{n{\bf k}}\Omega^{n, \gamma}_{\alpha\beta}({\bf k})\,.
  \label{eq:sbc_k_resolved}\tag{3}
\end{equation}
\begin{equation}
  \sigma^{{\rm SHC}, \gamma}_{\alpha\beta} = \frac{-e\hbar}{N_k\Omega_c}\sum_{\bf k}\Omega^{\gamma}_{\alpha\beta}({\bf k})
  \label{eq:shc_sbc}\tag{4}
\end{equation}

Therefore, where in the k-space contributes to the total SHC can be investigated using the k-resolved spin Berry curvature.

Here, we compute the spin Berry curvature again using the Ryoo method and the Qiao method.
Note that we pass the spin curren type as a `kwargs_formula` to the calculator, e.g. `kwargs_formula=dict(spin_current_type="ryoo")`.

In [12]:
from wannierberri import calculators as calc
calculators = {}
calculators["tabulate"] = calc.TabulatorAll(
     {
         "Energy": calc.tabulate.Energy(),
         'spin_berry_ryoo': calc.tabulate.SpinBerry(kwargs_formula=dict(spin_current_type="ryoo"), degen_thresh=1e-2),
         'spin_berry_qiao': calc.tabulate.SpinBerry(kwargs_formula=dict(spin_current_type="qiao"), degen_thresh=1e-2),
     },
    ibands=np.arange(system.num_wann),
    mode="path",
 )

calculator not described

calculator not described

calculator not described



NameError: name 'system' is not defined

In [13]:
path = wberri.Path(
    system,
    k_nodes=[
        [0.25, 0.75, 0.50], # W
        [0.50, 0.50, 0.50], # L
        [0.00, 0.00, 0.00], # Gamma
        [0.50, 0.00, 0.50], # X
        [0.50, 0.25, 0.75], # W
        [0.00, 0.00, 0.00], # Gamma
    ],
    labels=["W", "L", "$\Gamma$", "X", "W", "$\Gamma$"],
    length=600,
)

result_spin_berry = wberri.run(
    system,
    grid=path,
    calculators=calculators,
    parallel = parallel,
    print_Kpoints = False,
)

NameError: name 'system' is not defined

Now we sum over bands to compute the k-resolved spin Berry curvature:
$$\Omega^{\gamma}_{\alpha\beta}({\bf k}) = \sum_{n \in {\rm occ.}} \Omega^{n, \gamma}_{\alpha\beta}({\bf k})$$

In [14]:
nk = path.K_list.shape[0]
spin_berry_ryoo = np.zeros((nk, 3, 3, 3))
spin_berry_qiao = np.zeros((nk, 3, 3, 3))

for iband in range(system.num_wann):
    # Get the data for iband-th band
    e = result_spin_berry.results["tabulate"].get_data("Energy", iband)
    spin_berry_ryoo_nk = result_spin_berry.results["tabulate"].get_data("spin_berry_ryoo", iband)
    spin_berry_qiao_nk = result_spin_berry.results["tabulate"].get_data("spin_berry_qiao", iband)

    # Select k-point indices where the iband-th band is occupied
    inds_occupied = e < efermi
    
    # Add the spin Berry curvature of those bands
    spin_berry_ryoo[inds_occupied] += spin_berry_ryoo_nk[inds_occupied]
    spin_berry_qiao[inds_occupied] += spin_berry_qiao_nk[inds_occupied]

NameError: name 'path' is not defined

In [15]:
def get_signed_log10(x):
    return np.log10(abs(x)) * np.sign(x)
    
kline = path.getKline()
plt.plot(kline, get_signed_log10(spin_berry_ryoo[:, 0, 1, 2]), "k-", label='Ryoo')
plt.plot(kline, get_signed_log10(spin_berry_qiao[:, 0, 1, 2]), "r--", label='Qiao')

for i in path.labels.keys():
    plt.axvline(kline[i], c="k", lw=1)
plt.xticks([kline[i] for i in path.labels.keys()], path.labels.values())
plt.xlim([min(kline), max(kline)])
plt.axhline(0, c="k", lw=1)
plt.legend()
plt.title("$\mathrm{log}_{10} \Omega_\mathbf{k}$")

plt.show()

NameError: name 'path' is not defined

You can find that the spin Berry curvature calculated using the Qiao method shows more "wiggles" than the Ryoo method. This numerical difference has been first reported in [T. Ng et al, PRB 104 014412 (2021)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.104.014412):

>  It is worth
noting that there is jittering along Γ-Z, which occurs in the same path in WTe2 using the same method [22] (Qiao et al). However, such jittering disappears and the spin Berry curvature along Γ-Z becomes a smooth function using the method in Ref. [45] (Ryoo et al).

## Generating .sHu and .sIu from .mmn and .spn: mmn2uHu 

Even if you have not obtained .shu and .sIu from an ab initio code, you can make them from the overlap matrix and the spin matrix. Wannierberri provides the utility `wannierberri.utils.mmn2uHu`, which calculated the matrices `.uHu`, `.uIu`, `.sHu`, and/or `.sIu` from the `.mmn`, `.spn`, `.eig` matrices, and also reduces the number of bands in `.amn`, `.mmn`, `.eig`  and `.spn` files, by means of the sum-over-states formula

\begin{equation}
  \langle u_{m{\bf q}}\vert\hat{s}\hat{H}_{\bf q}\vert u_{n{\bf q}+\mathbf{b}}\rangle \approx \sum_l^{l_{\rm max}}  \left(s_{lm}({\bf q})\right)^* E_{l{\bf q}}   M_{ln}^{\mathbf{b}}({\bf q})\,.
\label{eq:sHu}\tag{5}
\end{equation}

\begin{equation}
  \langle u_{m{\bf q}}\vert\hat{s}\vert u_{n{\bf q}+\mathbf{b}}\rangle \approx \sum_l^{l_{\rm max}}  \left(s_{lm}({\bf q})\right)^*   M_{ln}^{\mathbf{b}}({\bf q})\,.
\label{eq:sIu}\tag{6}
\end{equation}

Here, $l_{\rm max}$ cannot exceed the number of bands included in the Wannier90 calculation (i.e. the `num_bands` parameter in `Pt.win`).

The `mmn2uHu` utility can be particularly useful when the calculation of `sHu` and `sIu` files are not implemented in the DFT code you are using.

In [16]:
from wannierberri.utils import mmn2uHu as mmn2uHu
os.chdir("data_Pt")
mmn2uHu.run_mmn2uHu(PREFIX="Pt", writeSHU=True, writeSIU=True, NBout=18, NBsum=24)

# For a command-line execution of the mmn2uHu utility, use
# python3 -m wannierberri.utils.mmn2uHu Pt NBout=18,NBsum=24,targets=sHu,sIu

# Rename sHu and sIu files
import shutil
shutil.move("reduced_NB=24/Pt_nbs=24.sHu", "reduced_NB=24/Pt.sHu")
shutil.move("reduced_NB=24/Pt_nbs=24.sIu", "reduced_NB=24/Pt.sIu")

# Copy chk and spn files
shutil.copyfile("Pt.chk", "reduced_NB=24/Pt.chk")
shutil.copyfile("Pt.spn", "reduced_NB=24/Pt.spn")

os.chdir("..")

FileNotFoundError: [Errno 2] No such file or directory: 'data_Pt'

Now, load the system object. Since all the data are in the `data_Pt/reduced_NB=24` directory, you should use the seedname `data_Pt/reduced_NB=24/Pt`.

In [17]:
system_mmn2uhu = wberri.System_w90("data_Pt/reduced_NB=24/Pt", berry=True, SHCryoo=True, SHCqiao=True)
system_mmn2uhu.set_structure([[0., 0., 0.]], ["Pt"])
system_mmn2uhu.set_symmetry_from_structure()

using fortio to read


FileNotFoundError: [Errno 2] No such file or directory: '/home/runner/work/wannier-berri-org/wannier-berri-org/source/tutorials/spin_hall/data_Pt/reduced_NB=24/Pt.chk'

### Problem: Compare the SHC computed using the sHu files from pw2wannier90 and mmn2uHu.

Compute the SHC using `system` and `system_mmn2uHu`.
Since the "Qiao" method does not use the sHu and sIu files, the two systems will give exactly the same result.
This may not be the case for the "Ryoo" method.

Use the keyword arguments given below (Fermi level scan at 0.026 meV ~ 300 K).

In [18]:
efermi_list = np.linspace(efermi - 1.0, efermi + 1.0, 101, True)

kwargs = dict(
    Efermi=efermi_list,
    omega=np.array([0.]),
    smr_fixed_width = 0.1, # Smearing for frequency in eV
    kBT = 0.026, # Smearing for Fermi level (Fermi-Dirac factor) in eV (not Kelvin)
)

# Fill in the code below.

NameError: name 'efermi' is not defined

## Further questions

If you are interested, try to answer the following questions:
- Try to converge the calculation using a different value of `smr_fixed_width`. In principle, to achieve an ideal convergence to the zero-smearing limit, one needs to first converge SHC increasing the grid size for a fixed `smr_fixed_width`, and then repeat the procedure with smaller `smr_fixed_width` until convergence.
- What happens if one include more bands in the NSCF calculation? Does the two methods converge to the same result? (To answer this question, one needs to perform additional DFT calculations.)